In [1]:
# Importar bibliotecas

import pandas as pd
import sqlite3
from pathlib import Path

In [2]:
# Leitura dos dados parquet

df_participantes = pd.read_parquet("../dados/raw/participantes-2024.parquet")
df_resultados = pd.read_parquet("../dados/raw/resultados-2024.parquet")

df_participantes.head()
df_resultados.head()	

,NU_SEQUENCIAL;NU_ANO;CO_ESCOLA;CO_MUNICIPIO_ESC;NO_MUNICIPIO_ESC;CO_UF_ESC;SG_UF_ESC;TP_DEPENDENCIA_ADM_ESC;TP_LOCALIZACAO_ESC;TP_SIT_FUNC_ESC;CO_MUNICIPIO_PROVA;NO_MUNICIPIO_PROVA;CO_UF_PROVA;SG_UF_PROVA;TP_PRESENCA_CN;TP_PRESENCA_CH;TP_PRESENCA_LC;TP_PRESENCA_MT;CO_PROVA_CN;CO_PROVA_CH;CO_PROVA_LC;CO_PROVA_MT;NU_NOTA_CN;NU_NOTA_CH;NU_NOTA_LC;NU_NOTA_MT;TX_RESPOSTAS_CN;TX_RESPOSTAS_CH;TX_RESPOSTAS_LC;TX_RESPOSTAS_MT;TP_LINGUA;TX_GABARITO_CN;TX_GABARITO_CH;TX_GABARITO_LC;TX_GABARITO_MT;TP_STATUS_REDACAO;NU_NOTA_COMP1;NU_NOTA_COMP2;NU_NOTA_COMP3;NU_NOTA_COMP4;NU_NOTA_COMP5;NU_NOTA_REDACAO
0,206403;2024;23052929;2301406;Aratuba;23;CE;2;1...
1,3604651;2024;42103770;4218004;Tijucas;42;SC;4;...
2,1461268;2024;;;;;;;;;4122404;Rolndia;41;PR;1;1...
3,4301058;2024;;;;;;;;;3534401;Osasco;35;SP;1;1;...
4,3148322;2024;21150354;2100436;Alto Alegre do M...


In [90]:
# Conexão com banco de dados
conn = sqlite3.connect("enem2024.db")
cursor = conn.cursor()

In [85]:
# Criar tabelas staging

df_participantes.to_sql("stg_participantes", conn, if_exists="replace", index=False)
df_resultados.to_sql("stg_resultados", conn, if_exists="replace", index=False)

4332944

In [70]:
# Exemplo para verificar o esquema da tabela
cursor.execute("PRAGMA table_info(stg_resultados);")
colunas = cursor.fetchall()
for coluna in colunas:
    print(coluna)

(0, 'NU_SEQUENCIAL;NU_ANO;CO_ESCOLA;CO_MUNICIPIO_ESC;NO_MUNICIPIO_ESC;CO_UF_ESC;SG_UF_ESC;TP_DEPENDENCIA_ADM_ESC;TP_LOCALIZACAO_ESC;TP_SIT_FUNC_ESC;CO_MUNICIPIO_PROVA;NO_MUNICIPIO_PROVA;CO_UF_PROVA;SG_UF_PROVA;TP_PRESENCA_CN;TP_PRESENCA_CH;TP_PRESENCA_LC;TP_PRESENCA_MT;CO_PROVA_CN;CO_PROVA_CH;CO_PROVA_LC;CO_PROVA_MT;NU_NOTA_CN;NU_NOTA_CH;NU_NOTA_LC;NU_NOTA_MT;TX_RESPOSTAS_CN;TX_RESPOSTAS_CH;TX_RESPOSTAS_LC;TX_RESPOSTAS_MT;TP_LINGUA;TX_GABARITO_CN;TX_GABARITO_CH;TX_GABARITO_LC;TX_GABARITO_MT;TP_STATUS_REDACAO;NU_NOTA_COMP1;NU_NOTA_COMP2;NU_NOTA_COMP3;NU_NOTA_COMP4;NU_NOTA_COMP5;NU_NOTA_REDACAO', 'TEXT', 0, None, 0)


In [104]:
# Armazena dados brutos exatamente como vieram do parquet

cursor.execute("""
DROP TABLE IF EXISTS stg_participantes;
""")

# Criar a tabela staging

cursor.execute("""
CREATE TABLE stg_participantes (
    NU_INSCRICAO INTEGER,
    NU_ANO INTEGER,
    TP_FAIXA_ETARIA INTEGER,
    TP_SEXO TEXT,
    TP_ESTADO_CIVIL INTEGER,
    TP_COR_RACA INTEGER,
    TP_NACIONALIDADE INTEGER,
    TP_ST_CONCLUSAO INTEGER,
    TP_ENSINO INTEGER,
    IN_TREINEIRO INTEGER,
    CO_MUNICIPIO_PROVA INTEGER,
    NO_MUNICIPIO_PROVA TEXT,
    CO_UF_PROVA INTEGER
);
""")

# Tabela staging para resultados
cursor.execute("""
DROP TABLE IF EXISTS stg_resultados;
""")

cursor.execute("""
CREATE TABLE stg_resultados (
    NU_SEQUENCIAL INTEGER,
    NU_ANO INTEGER,
    CO_ESCOLA INTEGER,
    CO_MUNICIPIO_ESC INTEGER,
    NO_MUNICIPIO_ESC TEXT,
    CO_UF_ESC INTEGER,
    SG_UF_ESC TEXT,
    TP_DEPENDENCIA_ADM_ESC INTEGER,
    CO_MUNICIPIO_PROVA INTEGER,
    NO_MUNICIPIO_PROVA TEXT,
    CO_UF_PROVA INTEGER,
    NU_NOTA_CN REAL,
    NU_NOTA_CH REAL,												
    NU_NOTA_LC REAL,
    NU_NOTA_MT REAL,	
    NU_NOTA_REDACAO REAL
);
""")


print("Tabelas staging criadas.")

Tabelas staging criadas.


In [ ]:
# Criar dimensões

cursor.executescript("""
DROP TABLE IF EXISTS dim_aluno;
CREATE TABLE dim_aluno AS
SELECT DISTINCT
    NU_INSCRICAO AS aluno_id,
    NU_ANO,
    TP_FAIXA_ETARIA,
    TP_SEXO,
    TP_ESTADO_CIVIL,
    TP_COR_RACA,
    TP_NACIONALIDADE,
    TP_ST_CONCLUSAO,
    TP_ENSINO,
    IN_TREINEIRO,
    CO_MUNICIPIO_PROVA,
    NO_MUNICIPIO_PROVA,
    CO_UF_PROVA
FROM stg_participantes;

DROP TABLE IF EXISTS dim_escola;
CREATE TABLE dim_escola AS
SELECT DISTINCT
    CO_ESCOLA AS escola_id,
    CO_MUNICIPIO_ESC,
    NO_MUNICIPIO_ESC,
    CO_UF_ESC,
    SG_UF_ESC,
    TP_DEPENDENCIA_ADM_ESC
FROM stg_resultados;

DROP TABLE IF EXISTS dim_local_prova;
CREATE TABLE dim_local_prova AS
SELECT DISTINCT
    CO_MUNICIPIO_PROVA AS municipio_id,
    NO_MUNICIPIO_PROVA,
    CO_UF_PROVA
FROM stg_resultados;

DROP TABLE IF EXISTS dim_tempo;
CREATE TABLE dim_tempo AS
SELECT DISTINCT
    NU_ANO AS ano
FROM stg_resultados;
""")

# Criar tabela fato

cursor.executescript("""
DROP TABLE IF EXISTS fato_resultados;
CREATE TABLE fato_resultados AS
SELECT
    r.NU_SEQUENCIAL AS resultado_id,
    p.NU_INSCRICAO AS aluno_id,
    r.CO_ESCOLA AS escola_id,
    r.CO_MUNICIPIO_PROVA AS municipio_id,
    NU_NOTA_CN REAL,
    NU_NOTA_CH REAL,												
    NU_NOTA_LC REAL,
    NU_NOTA_MT REAL,	
    NU_NOTA_REDACAO REAL
FROM stg_resultados r
JOIN stg_participantes p ON r.NU_ANO = p.NU_ANO
    AND r.CO_MUNICIPIO_PROVA = p.CO_MUNICIPIO_PROVA;
""")

print("Dimensões e tabela fato criadas.")

Dimensões e tabela fato criadas.


### 1. Qual a escola com a maior média de notas?

In [116]:
q1 = """
SELECT e.NO_MUNICIPIO_ESC, e.SG_UF_ESC, e.TP_DEPENDENCIA_ADM_ESC,
       AVG((f.NU_NOTA_1 + f.NU_NOTA_2 + f.NU_NOTA_3)/3) AS media_escola
FROM fato_resultados f
JOIN dim_escola e ON f.escola_id = e.escola_id
GROUP BY e.NO_MUNICIPIO_ESC, e.SG_UF_ESC, e.TP_DEPENDENCIA_ADM_ESC
ORDER BY media_escola DESC
LIMIT 1;
"""

### 2. Qual o aluno com a maior média de notas?

In [118]:
q2 = """
SELECT a.aluno_id,
       AVG((f.NU_NOTA_1 + f.NU_NOTA_2 + f.NU_NOTA_3)/3) AS media_aluno
FROM fato_resultados f
JOIN dim_aluno a ON f.aluno_id = a.aluno_id
GROUP BY a.aluno_id
ORDER BY media_aluno DESC
LIMIT 1;
"""

### 3. Qual a média geral?

In [119]:
q3 = """
SELECT AVG((NU_NOTA_1 + NU_NOTA_2 + NU_NOTA_3)/3) AS media_geral
FROM fato_resultados;
"""

### 4. Qual o porcentagem %  de ausências?

In [120]:
# (Supondo que ausência = todas notas NULL)

q4 = """
SELECT 
    (SUM(CASE WHEN NU_NOTA_1 IS NULL AND NU_NOTA_2 IS NULL AND NU_NOTA_3 IS NULL THEN 1 ELSE 0 END) * 100.0) / COUNT(*) 
    AS perc_ausentes
FROM fato_resultados;
"""

### 5. Número total de inscritos?

In [121]:
q5 = "SELECT COUNT(DISTINCT aluno_id) AS total_inscritos FROM fato_resultados;"

### 6. Média por disciplina

In [122]:
q6 = """
SELECT AVG(NU_NOTA_1) AS media_nota1,
       AVG(NU_NOTA_2) AS media_nota2,
       AVG(NU_NOTA_3) AS media_nota3
FROM fato_resultados;
"""

### 7. Média por Sexo?

In [123]:
q7 = """
SELECT a.TP_SEXO,
       AVG((f.NU_NOTA_1 + f.NU_NOTA_2 + f.NU_NOTA_3)/3) AS media_sexo
FROM fato_resultados f
JOIN dim_aluno a ON f.aluno_id = a.aluno_id
GROUP BY a.TP_SEXO;
"""

### 8. Média por Etnia

In [ ]:
q8 = """
SELECT a.TP_COR_RACA,
       AVG((f.NU_NOTA_1 + f.NU_NOTA_2 + f.NU_NOTA_3)/3) AS media_etnia
FROM fato_resultados f
JOIN dim_aluno a ON f.aluno_id = a.aluno_id
GROUP BY a.TP_COR_RACA;
"""